In [2]:
# prompt: pip install wandb

!pip install wandb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.3 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
wandb_key = userdata.get('WANDB_KEY')

In [4]:
# prompt: Connect to wandb using the key stored in the variable wandb_key and return list of runs

import wandb
wandb.login(key = wandb_key)



wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
#list names of projects by common-sense-makers
entity="common-sense-makers"
projects = wandb.api.list_projects(entity=entity)
for project in projects:
    print(project["name"])

pytorch-intro
basic-intro
st-app-multi-v1
st-demo-sandbox
st_demo-multi_v1_testing
st_demo-multi_v1
data_aggregation
st_demo-v0.2
st_demo-v0.1
testing


In [23]:
project = "st-app-multi-v1"




In [ ]:
#log data as json
import wandb
import pandas as pd

# Assuming you have a DataFrame named df with your data
df = pd.read_csv("/content/st_data.csv")

wandb.init(project="testing")
artifact = wandb.Artifact("labeled_data_v0", type="dataset")

# Save the DataFrame to a JSON file
json_path = "/content/st_data.json"
df.to_json(json_path, orient="records")

# Add the JSON file to the artifact
artifact.add_file(json_path, name="st_data.json")

# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()


In [ ]:
#log dataset as wandb.Table object
#which is also JSON but has nice display
import wandb
import pandas as pd

# Assuming you have a DataFrame named df with your data
df = pd.read_csv("/content/st_data_v1.csv")

wandb.init(project="testing")
artifact = wandb.Artifact("labeled_data_v0", type="dataset")

# Create a wandb.Table from the Pandas DataFrame
table = wandb.Table(dataframe=df)

# Add the wandb.Table to the artifact
artifact.add(table, "labeled_data_table")

# Log the artifact
wandb.log_artifact(artifact, aliases=["latest"])

wandb.run.finish()


This block imports a table from an artifact, whose path we must specify, and saves only the 'Text' and 'True Label' filed in a pd data frame. So later, we can evaluate predictions of models over the text entries.

In [22]:
#download a table from datasetartifact
#save it in a dataframe
path = "common-sense-makers/testing/labeled_data_v0:v4"

import wandb
from pathlib import Path
import json
import pandas as pd

api = wandb.Api()
artifact = api.artifact(path)

#download path to table
a_path = artifact.download()
table_path = Path(f"{a_path}/labeled_data_table.table.json")
raw_data = json.load(table_path.open())

#put it in a dataframe
x = raw_data["columns"].index('Text')
y = raw_data["columns"].index('True Label')

rows = []
i = 0
exception = 1
while exception:
  try:
    #assume x<y
    row_d = dict(zip(["Text",'True Label'],
                     [raw_data["data"][i][x],raw_data["data"][i][y]]))
    #print(row_d)
    rows.append(row_d)
    i=i+1
    exception = 1
  except:
    print("exceprion")
    exception = 0


df = pd.DataFrame(rows)

print(df)


wandb:   1 of 1 files downloaded.  


exceprion
                                                 Text  \
0   I just did the proofs for our "Epistemology of...   
1                             I want to read this!!!!   
2                                I want to read this!   
3   I just did the proofs for our "Epistemology of...   
4   I just did the proofs for our "Epistemology of...   
..                                                ...   
73  Good thread and paper I'm looking forward to r...   
74  Good thread and paper I'm looking forward to r...   
75  For a long time political philosophers ignored...   
76  For a long time political philosophers ignored...   
77  For a long time political philosophers ignored...   

                             True Label  
0                                 Other  
1        ['read', 'other', 'other:ggg']  
2       ['read', 'other', 'other:blah']  
3   ['announce', 'other', 'other:test']  
4                          ['announce']  
..                                  ...  
73         

Create CSV out of all prediction artifacts. Faster than the old script.

In [ ]:
#Select which project
project = "st-app-multi-v1"

In [35]:
import wandb
import pandas as pd
import os
import json
from pathlib import Path

# Initialize wandb
wandb.init()
api = wandb.Api()
# Specify the project and artifact type
project_name = project
TYPE = 'prediction'

# Retrieve all artifacts of the specified type

artifacts = []

collections = [
      coll for coll in api.artifact_type(type_name=TYPE, project=project_name).collections()
  ]


for coll in collections:
    for artifact in coll.artifacts():
        artifacts += [artifact]


# Process artifacts and create a CSV file
csv_data = []
rows = []
for artifact in artifacts:
    # Access the files in the artifact
    a_path = artifact.download()

    table_path = Path(f"{a_path}/predictions.table.json")
    raw_data = json.load(table_path.open())

    row_d = dict(zip(raw_data['columns'],
                     raw_data["data"][0]))
    #print(row_d)
    rows.append(row_d)

df = pd.DataFrame(rows)

print(df)

csv_data.append(df)

# Concatenate all dataframes into one
final_dataframe = pd.concat(csv_data, ignore_index=True)

# Save the concatenated dataframe to a CSV file
output_csv_path = 'output_data.csv'
final_dataframe.to_csv(output_csv_path, index=False)

print(f"CSV file saved at {output_csv_path}")

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downlo

                                                 User  \
0                                                 rtk   
1                                        Nick Vincent   
2                                        Nick Vincent   
3                    p(Alex Kiefer | everything else)   
4                                       Abeba Birhane   
5                                         Nils Gilman   
6                                      Tania Lombrozo   
7                                  Elizabeth McDaniel   
8                                   Kording —-& Lab 🦖   
9                                    Zr. Nabu Kudurru   
10                                 J. Nathan Matias 🦣   
11  rtk - https://twitter.com/ChombaBupe/status/17...   
12                                         Tim Howles   
13                               ♻️ Leshem Choshen ♻️   
14  rtk - https://mastodon.social/@jonny@neuromatc...   
15                                   Mx. Aria Stewart   
16   Filip Piekniewski🌻 🐘:@fili